# Part 1 - Data Preparation and Preprocessing [8 Points]


In [2]:
# IMPORT MODULES
import pandas as pd
import re

In [3]:
# Perform data preparation and processing
## code
orig_df = pd.read_csv("data.csv")
orig_df.head()

C:\Users\andre\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
1,37404337,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,NaN,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,NaN,0,30,NaN
2,37404356,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,NaN,0,30,NaN
3,37404330,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,NaN,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,NaN,0,30,NaN
4,37404308,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",NaN,0,30,NaN


In [4]:
# drop id, drop duplicates
orig_df = orig_df.drop_duplicates()
orig_df = orig_df.drop(columns='Id')
orig_df.head()

In [9]:
# take job type from title put in job type column where job type is NaN
jTypes = ['Casual', 'Part Time', 'Full Time']
edited = 0

for index, row in orig_df.iterrows():
    for job in jTypes:
        match = re.match(job, row['Title'], re.IGNORECASE)
        
        if match is not None:
            # Replace the job type from the title
            orig_df.at[index, 'Title'] = row['Title'].replace(match.group(), '')
            
            # If the row has no job type add it in
            if pd.isnull(row['JobType']):
                if job == 'Casual':
                    orig_df.at[index, 'JobType'] = 'Casual/Vacation'
                else:
                    orig_df.at[index, 'JobType'] = job
            edited += 1
orig_df[['Title', 'JobType']].head()

,Title,JobType
0,Stock Replenisher,Casual/Vacation
1,Stock Replenisher,Casual/Vacation
2,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,NaN
3,Team member - Belrose,NaN
4,"Business Banking Contact Centre Specialist, Ni...",NaN


In [11]:
# create separate data frames
job_type_df = orig_df[['SubClassification', 'JobType']].copy()
job_type_df = job_type_df.dropna(axis=0, subset=['JobType'])
class_subclass_df = orig_df[['SubClassification', 'Classification', 'Location']].copy()
class_subclass_df.dropna()
# can talk about how actual salary and full description different
salary_df = orig_df[['SubClassification', 'Classification', 'LowestSalary', 'HighestSalary']].copy()
job_type_df.head()

,SubClassification,JobType
0,Retail Assistants,Casual/Vacation
1,Retail Assistants,Casual/Vacation
6,Waiting Staff,Casual/Vacation
7,Waiting Staff,Casual/Vacation
9,Kitchen & Sandwich Hands,Casual/Vacation


### 1) Describe the dataset


#### 1.1) What are the categories/domains of the dataset?

#### 1.2) What is dataset structure/format

#### 1.3) What are attributes/features of the data are you going to use?

#### 1.4) Which parts of the dataset will you use or all of them?


### 2) Describe the steps you used for data preparation and preprocessing

[2-3 Paragraphs] 
#### 2.1. How did we load the data?
#### 2.2. How did we normalize the data?
#### 2.3. How did we clean the data?
- Can talk about initial insights about $ salary differences in data and job type inconsistences

### 3) What is your hypothesis about the analysis outcome?

[1-2 paragraphs] This our hypothesis

# Part 2 - Data Analysis and Interpretation [17 points]

- Perform exploratory data analysis
- Perform statistical data analysis
- Perform predictive data analysis

In [12]:
## CODE
## Perform predictive on where the job market will be in x years

### 1) Study the job metadata. Extract the relevant information to describe the job's attributes.

#### 1.1 What is the sector, sub-sector of each job?

#### 1.2 Where is the location of the job?
// maybe CBD vs inner suburbs vs other
#### 1.3 Which is the range of salaries for each job? 

[1-2 paragraphs]

### 2) Study the job market by locations

#### 2.1 What is the market size in each city? Which are the hottest job sectors in each city?



#### 2.2 Which range of salary is common on each city? Where are the employees more well-paid?

#### 2.3 Can you detect the pattern of posting: e.g. are more jobs posted at the beginning of the month?

[1-2 paragraphs]

### 3) Study the market by sectors 

#### 3.1 Which sectors keep the highest market share?

#### 3.2 In each sector, which sub-sectors are the main spotlights?

#### 3.3 What is the salary range for each sector/sub-sector? Can you compare salary range between sectors/subsectors?

#### 3.4 What is the trending of market i.e. if a high school student asked you which subject should he/she learn in university (to guarantee a job in the future), what is your advice?

#### 3.5 Can you detect which skills are required in each sector?

### 4) Visualize the results on an interactive web page
- Trend analysis: visualize number of jobs by locations, by sectors, etc
- Compare between locations or sectors about the number of jobs, the salary, etc.
- Present the necessary skills by sectors, by subsectors

#### 5) Using PySpark for your data analysis [2 points]

# Part 3 - Evaluation [5 Points]

### Part 1) What are the findings of your data analytics?
[2-3 paragraphs, 2 points]

### Part 2) What actions for balancing the markets do you suggest based on your findings?
[1-2 paragraphs, 1 point]

### Part 3) How could you refine your data analytics? [1-2 paragraphs]

#### 3.1 Could you use different data sources?

#### 3.2 Could you choose different parameters?

#### 3.3 Could you choose other techniques?

#### 3.4 Can you think of ways to obtain more relevant data?

### Part 4) Are there any implications for employers and employees based on the findings you obtained? Justify answer. [1-2 paragraphs]
- Answer

#### Part 5) Present and visualize your data story on an online Web Page [OPTIONAL - up to 5 bonus points]
- Publish on Google Cloud or github.io